In [ ]:
import lpips
import torch
import torchvision.transforms as transforms
from skimage.metrics import structural_similarity as ssim_skimage
from skimage.metrics import peak_signal_noise_ratio as psnr
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import re

Configuring directories

In [ ]:
distorted_images_dir = "Distortions_v2"
csv_output_dir = "CSVs"
chart_output_dir = "Charts"

Measuring values and saving to csv file

In [ ]:
# Initialize LPIPS model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_fn = lpips.LPIPS(net='alex').to(device)

# Transform: convert image to tensor and normalize to [-1, 1]
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Optional: adjust to match your data
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # LPIPS expects input in [-1, 1]
])

results = []

# Loop over picture folders (Degus, Hotdog, etc.)
for scene_folder in os.listdir(distorted_images_dir):
    scene_path = os.path.join(distorted_images_dir, scene_folder)
    if not os.path.isdir(scene_path):
        continue

    # Loop over distortion type folders (Blur, Noise, etc.)
    for distortion_folder in os.listdir(scene_path):
        distortion_path = os.path.join(scene_path, distortion_folder)
        if not os.path.isdir(distortion_path):
            continue

        # Find ref image (ends with 0.bmp and is only .bmp file)
        reference_image_path = None
        for file in os.listdir(distortion_path):
            if file.endswith('_0.bmp'):
                reference_image_path = os.path.join(distortion_path, file)
                break
        if reference_image_path is None:
            print(f"No reference image found in {scene_path}")
            continue
        ref_img = transform(Image.open(reference_image_path).convert('RGB')).unsqueeze(0).to(device)

        # Calculate lpips for each picture
        for file in os.listdir(distortion_path):
            if not file.endswith('.bmp') and not file.endswith('.png'):
                continue

            distorted_image_path = os.path.join(distortion_path, file)

            # Load and preprocess distorted image
            dist_img = transform(Image.open(distorted_image_path).convert('RGB')).unsqueeze(0).to(device)

            # Compute LPIPS distance
            with torch.no_grad():
                distance_lpips = loss_fn(ref_img, dist_img).item()

            # ref_rgb = cv2.cvtColor(ref_cv, cv2.COLOR_BGR2RGB)
            # dist_rgb = cv2.cvtColor(dist_cv, cv2.COLOR_BGR2RGB)
            # Compute SSIM
            distance_ssim, _ = ssim_skimage(
                cv2.cvtColor(cv2.imread(reference_image_path), cv2.COLOR_BGR2RGB),
                cv2.cvtColor(cv2.imread(distorted_image_path), cv2.COLOR_BGR2RGB),
                channel_axis=2,
                full=True
            )
            # Compute PSNR value
            distance_psnr = psnr(cv2.imread(reference_image_path), cv2.imread(distorted_image_path), data_range=255)

            #extract distortion value
            distortion_value = -1
            match = re.search(r'^([^_]+)_([^_]+)_(.*)\.(png|bmp)$', file)
            try:
                distortion_value = float(match.group(3).replace('_', '.'))
            except:
                print(f"No distortion value found in {file}")

            # Save result
            results.append({
                'Picture': scene_folder,
                'DistortionType': distortion_folder,
                'DistortionValue': distortion_value,
                'Filename': file,
                'Method': 'LPIPS',
                'Value': distance_lpips
            })
            results.append({
                'Picture': scene_folder,
                'DistortionType': distortion_folder,
                'DistortionValue': distortion_value,
                'Filename': file,
                'Method': 'SSIM',
                'Value': distance_ssim
            })
            results.append({
                'Picture': scene_folder,
                'DistortionType': distortion_folder,
                'DistortionValue': distortion_value,
                'Filename': file,
                'Method': 'PSNR',
                'Value': distance_psnr
            })

# Save all results to CSV
df = pd.DataFrame(results)
os.makedirs(csv_output_dir, exist_ok=True)
df.to_csv(csv_output_dir + '/distortion_results.csv', index=False)
print("✅ Results saved to lpips_distortion_results.csv")

Creating charts

In [ ]:
# === Load CSV ===
df = pd.read_csv(csv_output_dir + "/distortion_results.csv")

# Add synthetic LPIPS=0 for level 0 reference images
reference_rows = []
for picture in df['Picture'].unique():
    for distortion in df['DistortionType'].unique():
        reference_rows.append({
            'Picture': picture,
            'DistortionType': distortion,
            'DistortionValue': 0,
            'Filename': 'image_0.bmp',
            'Method': 'LPIPS',
            'Value': 0.0
        })

df = pd.concat([df, pd.DataFrame(reference_rows)], ignore_index=True)
# === Plotting ===
os.makedirs(chart_output_dir, exist_ok=True)

for method in df['Method'].unique():
    method_df = df[df['Method'] == method]
    for distortion in df['DistortionType'].unique():
        plt.figure(figsize=(8, 5))
        distortion_df = method_df[method_df['DistortionType'] == distortion]

        for picture in sorted(distortion_df['Picture'].unique()):
            picture_df = distortion_df[distortion_df['Picture'] == picture]
            picture_df = picture_df.sort_values('DistortionValue')

            # Build y and x values
            x = picture_df['DistortionValue']
            y = picture_df['Value']
            plt.plot(x, y, marker='o', label=picture)

        plt.title(f'{method} vs Distortion Value – {distortion}')
        plt.xlabel('Distortion Parameter')
        plt.ylabel(f'{method} Distance')
        plt.legend(title='Picture', fontsize='small')
        plt.grid(True)
        plt.tight_layout()
        os.makedirs(f'{chart_output_dir}/{method}', exist_ok=True)
        plt.savefig(f'{chart_output_dir}/{method}/{distortion}_{method}_chart.png')
        plt.close()

print("✅ All charts saved")

Creating group chart

In [ ]:
df = pd.read_csv(csv_output_dir + "/distortion_results.csv")

# Keep rows with the highest DistortionValue for each Picture & DistortionType
idx = df.groupby(['Picture', 'DistortionType'])['DistortionValue'].idxmax()
df_max_distortions = df.loc[idx]
df_max_distortions['DistortionLabel'] = df_max_distortions.apply(lambda row: f"{row['DistortionType']} ({row['DistortionValue']})", axis=1)

# Pivot for grouped bar chart: index=DistortionType, columns=Picture, values=LPIPS
pivot_df = df_max_distortions.pivot(index='DistortionLabel', columns='Picture', values='Value')

# Plot
pivot_df.plot(kind='bar', figsize=(10, 6))

plt.ylabel('LPIPS Distance')
plt.title('LPIPS at Max Distortion per Picture per Distortion Type')
plt.xticks(rotation=45)
plt.legend(title='Picture')
plt.tight_layout()
plt.savefig(f'{chart_output_dir}/lpips_distortion_max.png')
plt.close()
